In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import tqdm
import re
from sklearn.multioutput import MultiOutputRegressor
import seaborn as sns
import matplotlib.pyplot as plt

from warnings import simplefilter

import matplotlib.pyplot as plt
import lightgbm as lgb

from statsmodels.tsa.deterministic import DeterministicProcess, CalendarFourier
from sklearn.model_selection import TimeSeriesSplit
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import os

In [2]:
# Set Matplotlib defaults
plt.rc("figure", autolayout=True, figsize=(11, 4))
plt.rc(
    "axes",
    labelweight="bold",
    labelsize="large",
    titleweight="bold",
    titlesize=16,
    titlepad=10,
)
plot_params = dict(
    color="0.75",
    style=".-",
    markeredgecolor="0.25",
    markerfacecolor="0.25",
)
%config InlineBackend.figure_format = 'retina'


def plot_multistep(y, every=1, ax=None, palette_kwargs=None):
    palette_kwargs_ = dict(palette='husl', n_colors=16, desat=None)
    if palette_kwargs is not None:
        palette_kwargs_.update(palette_kwargs)
    palette = sns.color_palette(**palette_kwargs_)
    if ax is None:
        fig, ax = plt.subplots()
    ax.set_prop_cycle(plt.cycler('color', palette))
    for date, preds in y[::every].iterrows():
        preds.index = pd.period_range(start=date, periods=len(preds))
        preds.plot(ax=ax)
    return ax

In [3]:
train_data = pd.read_csv('../data/sales_train_evaluation.csv')
calendar = pd.read_csv('../data/calendar.csv')
price = pd.read_csv('../data/sell_prices.csv')

In [4]:
train_data

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,4,0,0,0,0,3,3,0,1
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,1,2,1,1,0,0,0,0,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,2,0,0,0,2,3,0,1
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,1,0,4,0,1,3,0,2,6
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,2,1,0,0,2,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_evaluation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,0,0,2,2,...,1,0,3,0,1,1,0,0,1,1
30486,FOODS_3_824_WI_3_evaluation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
30487,FOODS_3_825_WI_3_evaluation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,0,6,0,2,...,0,0,1,2,0,1,0,1,0,2
30488,FOODS_3_826_WI_3_evaluation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,1,1,1,4,6,0,1,1,1,0


In [5]:
melt_df = pd.melt(train_data, id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], 
        value_vars=[f'd_{i}' for i in range(1,1942)], value_name='sales')
melt_df

,id,item_id,dept_id,cat_id,store_id,state_id,variable,sales
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
...,...,...,...,...,...,...,...,...
59181085,FOODS_3_823_WI_3_evaluation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,d_1941,1
59181086,FOODS_3_824_WI_3_evaluation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,d_1941,0
59181087,FOODS_3_825_WI_3_evaluation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,d_1941,2
59181088,FOODS_3_826_WI_3_evaluation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,d_1941,0


In [6]:
# target encoding
melt_df['item_mean'] = melt_df.groupby('id')['sales'].rolling(14).mean().reset_index(level=0, drop=True).astype(np.float16)
melt_df['item_std'] = melt_df.groupby('id')['sales'].rolling(14).std().reset_index(level=0, drop=True).astype(np.float16)
melt_df

,id,item_id,dept_id,cat_id,store_id,state_id,variable,sales,item_mean,item_std
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,NaN,NaN
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,NaN,NaN
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,NaN,NaN
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,NaN,NaN
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
59181085,FOODS_3_823_WI_3_evaluation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,d_1941,1,0.643066,0.841797
59181086,FOODS_3_824_WI_3_evaluation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,d_1941,0,0.142822,0.363037
59181087,FOODS_3_825_WI_3_evaluation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,d_1941,2,1.000000,1.109375
59181088,FOODS_3_826_WI_3_evaluation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,d_1941,0,1.428711,1.650391


In [7]:
day_columns = [column for column in train_data.columns if 'd_' in column]
groups = melt_df[['cat_id', 'store_id']].drop_duplicates().values
len(groups)

30

In [8]:
groups

array([['HOBBIES', 'CA_1'],
       ['HOUSEHOLD', 'CA_1'],
       ['FOODS', 'CA_1'],
       ['HOBBIES', 'CA_2'],
       ['HOUSEHOLD', 'CA_2'],
       ['FOODS', 'CA_2'],
       ['HOBBIES', 'CA_3'],
       ['HOUSEHOLD', 'CA_3'],
       ['FOODS', 'CA_3'],
       ['HOBBIES', 'CA_4'],
       ['HOUSEHOLD', 'CA_4'],
       ['FOODS', 'CA_4'],
       ['HOBBIES', 'TX_1'],
       ['HOUSEHOLD', 'TX_1'],
       ['FOODS', 'TX_1'],
       ['HOBBIES', 'TX_2'],
       ['HOUSEHOLD', 'TX_2'],
       ['FOODS', 'TX_2'],
       ['HOBBIES', 'TX_3'],
       ['HOUSEHOLD', 'TX_3'],
       ['FOODS', 'TX_3'],
       ['HOBBIES', 'WI_1'],
       ['HOUSEHOLD', 'WI_1'],
       ['FOODS', 'WI_1'],
       ['HOBBIES', 'WI_2'],
       ['HOUSEHOLD', 'WI_2'],
       ['FOODS', 'WI_2'],
       ['HOBBIES', 'WI_3'],
       ['HOUSEHOLD', 'WI_3'],
       ['FOODS', 'WI_3']], dtype=object)

In [9]:
def make_fourier(df):
    fourier = CalendarFourier(freq='M', order=4)

    dp = DeterministicProcess(
        index=df.index,
        order=1,
        constant=True,
        seasonal=True,
        drop=True,   
        additional_terms=[fourier]
        )

    time_features = dp.in_sample()
    return time_features



def make_lags(df, lags):
    return pd.DataFrame(
        {
            f'y_lag_{i}': df.groupby('id').sales.shift(i) for i in range(1, lags+1)
        }
    )

def make_rolling_mean(df, lags, window):
        return pd.DataFrame(
        {
            f'y_rolling_mean_{i}': df.groupby('id').sales.shift(i).rolling(window).mean() for i in range(1, lags+1)
        }
    )

def make_rolling_std(df, lags, window):
        return pd.DataFrame(
        {
            f'y_rolling_std_{i}': df.groupby('id').sales.shift(i).rolling(window).std() for i in range(1, lags+1)
        }
    )
    
def make_multistep_target(df, steps):
    return pd.DataFrame(
        {f'F{i}': df.groupby('id').sales.shift(-i)
         for i in range(1, steps+1)})

def make_price_feature(ts):
    return pd.DataFrame(
        {
            f'price_lag_1': ts.shift(1),
            f'price_lag_7': ts.shift(7),
            f'price_change_ratio': ts/ts.shift(1),
            f'price_change_ratio7': ts/ts.shift(7),
            f'price_rolling_mean_7': ts.rolling(7).mean(),
            f'price_rolling_std_7': ts.rolling(7).std(),
            f'price_rolling_max_7': ts.rolling(7).max(),
            f'price_rolling_max_14': ts.rolling(14).max(),
            f'price_rolling_min_7': ts.rolling(7).min(),
            f'price_rolling_min_14': ts.rolling(14).min()
            
        })

In [10]:
for cat_id, store_id in groups:
    selected_items = melt_df[(melt_df['cat_id']==cat_id) & (melt_df['store_id']==store_id)]
    
    merged_item_sales = selected_items.merge(calendar, how='left', left_on='variable', right_on='d')[['date', 'sales', 'item_mean', 'item_std', 'variable', 'id', 'store_id', 'item_id', 'wm_yr_wk']]

    merged_item_sales['date'] = pd.to_datetime(merged_item_sales['date'])
    merged_item_sales.set_index('date', inplace=True)
    merged_item_sales.index = merged_item_sales.index.to_period('D')
    
    merged_item_sales['sell_price'] = merged_item_sales.merge(price, on=['store_id', 'item_id', 'wm_yr_wk'], how='left')['sell_price'].values
    
    time_features = make_fourier(merged_item_sales)
    
    lag_features = make_lags(merged_item_sales, lags=14)
    lag_features = lag_features.fillna(0.0)

    rolling_mean_features = make_rolling_mean(merged_item_sales, lags=14, window=7)
    rolling_mean_features = rolling_mean_features.fillna(0.0)

    rolling_std_features = make_rolling_std(merged_item_sales, lags=14, window=7)
    rolling_std_features = rolling_std_features.fillna(0.0)
    
    price_features = make_price_feature(merged_item_sales.sell_price)
    price_features = price_features.fillna(0)
    
    all_data = pd.concat([time_features, lag_features, rolling_mean_features, rolling_std_features, price_features], axis=1)
    all_data['sales'] = merged_item_sales['sales']
    all_data['item_mean'] = merged_item_sales['item_mean']
    all_data['item_std'] = merged_item_sales['item_std']
    all_data['id'] = merged_item_sales['id']
    all_data['trend'] = merged_item_sales.variable.apply(lambda x: x.replace('d_', '')).astype(np.float16)
    all_data = all_data.fillna(0)
    all_data.rename({name: re.sub(r'[^a-zA-Z0-9_]', '_', name) for name in all_data.columns}, axis=1, inplace=True)

    all_data.to_csv(f'../data/cat_and_store_data/{cat_id}_and_{store_id}_features.csv', index=False)
    
    y = make_multistep_target(all_data, steps=28).dropna()

    y.to_csv(f'../data/cat_and_store_data/{cat_id}_and_{store_id}_target.csv', index=False)
    
    del selected_items, merged_item_sales, all_data, y
    
    print(f'{cat_id}_{store_id} done.')

HOBBIES_CA_1 done.
HOUSEHOLD_CA_1 done.
FOODS_CA_1 done.
HOBBIES_CA_2 done.
HOUSEHOLD_CA_2 done.
FOODS_CA_2 done.
HOBBIES_CA_3 done.
HOUSEHOLD_CA_3 done.
FOODS_CA_3 done.
HOBBIES_CA_4 done.
HOUSEHOLD_CA_4 done.
FOODS_CA_4 done.
HOBBIES_TX_1 done.
HOUSEHOLD_TX_1 done.
FOODS_TX_1 done.
HOBBIES_TX_2 done.
HOUSEHOLD_TX_2 done.
FOODS_TX_2 done.
HOBBIES_TX_3 done.
HOUSEHOLD_TX_3 done.
FOODS_TX_3 done.
HOBBIES_WI_1 done.
HOUSEHOLD_WI_1 done.
FOODS_WI_1 done.
HOBBIES_WI_2 done.
HOUSEHOLD_WI_2 done.
FOODS_WI_2 done.
HOBBIES_WI_3 done.
HOUSEHOLD_WI_3 done.
FOODS_WI_3 done.
